In [1]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
#from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier


from sklearn.model_selection import StratifiedKFold

from sklearn.metrics import classification_report, confusion_matrix, precision_score, recall_score, f1_score, cohen_kappa_score, \
                            precision_recall_curve, average_precision_score, roc_auc_score, roc_curve, auc, accuracy_score, matthews_corrcoef

from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.utils.class_weight import compute_class_weight
from sklearn import preprocessing

from sklearn.exceptions import ConvergenceWarning
import warnings

from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler 

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

from pathlib import Path

#import shap

import xgboost
#import lightgbm as lgb



from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier


import time

# load JS visualization code to notebook
#shap.initjs()

# dataset params

In [2]:
# path to the data file
data_file_loc = Path('./')

# load the dataset into dataframe
raw_df = pd.read_csv(data_file_loc/'data/SHAP_150.csv')
raw_df.head()


AAA       GGG       AGT         C      CCCC        AC        TA  \
0  0.014101  0.020564  0.008813  0.256455  0.004115  0.056371  0.032296   
1  0.011680  0.033402  0.013525  0.264031  0.009633  0.045892  0.033190   
2  0.021417  0.011532  0.006590  0.292282  0.003300  0.057566  0.023026   
3  0.025397  0.015238  0.009524  0.227647  0.001906  0.048858  0.043782   
4  0.012591  0.021308  0.012107  0.267537  0.005814  0.047919  0.031946   

   pseknc.41       GGC       ACG  ...      ACCC      CTTT      GCCA      CCGA  \
0      0.014  0.024677  0.008813  ...  0.004703  0.004115  0.008818  0.000000   
1      0.014  0.023566  0.003074  ...  0.004304  0.006969  0.008198  0.001640   
2      0.013  0.021417  0.006590  ...  0.008251  0.001650  0.008251  0.003300   
3      0.007  0.012063  0.001905  ...  0.002541  0.005083  0.005718  0.000000   
4      0.016  0.027119  0.005811  ...  0.003876  0.005814  0.007267  0.002422   

        CTC       GTC      TGCC  pseknc.27      CACC  location  
0  0.015864  0.012338  0.002939      0.003  0.004703         0  
1  0.023156  0.013730  0.007584      0.002  0.005124         0  
2  0.019769  0.008237  0.004950      0.008  0.008251         0  
3  0.017778  0.013333  0.003812      0.002  0.003812         0  
4  0.017918  0.014044  0.007752      0.004  0.003876         0  

[5 rows x 151 columns]

In [3]:
!ls "$data_file_loc"

ADNI_clinical_NestedCV-Copy1.ipynb  mRNA_loc_mclass_NestedCV.ipynb
data				    mRNA_loc_OVA_NestedCV.ipynb


In [4]:
# features + target
col_names = list(raw_df.columns)
print(col_names)

['AAA', 'GGG', 'AGT', 'C', 'CCCC', 'AC', 'TA', 'pseknc.41', 'GGC', 'ACG', 'pseknc.21', 'GACG', 'pseknc.11', 'GG', 'AAAA', 'A', 'AAT', 'GCCC', 'CCCA', 'pseknc.0', 'CAG', 'pseknc.37', 'CTA', 'CC', 'CGAT', 'pseknc.65', 'ACAG', 'CCC', 'ATT', 'AA', 'pseknc.15', 'pseknc.42', 'AT', 'AAG', 'AG', 'pseknc.70', 'CTCC', 'AATT', 'CGAA', 'GAC', 'G', 'CGCG', 'GGAG', 'GA', 'CCT', 'AATG', 'CGT', 'pseknc.33', 'AGTG', 'GC', 'GGCG', 'CCGC', 'GGT', 'GAGC', 'GCAG', 'pseknc.18', 'AATA', 'GTCT', 'ATTG', 'pseknc.38', 'GCT', 'GGGG', 'ACA', 'pseknc.24', 'CGTT', 'ATTT', 'TCG', 'GAAG', 'TGT', 'CAC', 'CGA', 'CCTG', 'ACGA', 'GGGC', 'TTC', 'CT', 'pseknc.68', 'AACG', 'TGC', 'ATTA', 'TTCG', 'pseknc.73', 'GAAC', 'pseknc.9', 'GTGA', 'AGA', 'GGTA', 'GCA', 'CAGA', 'CCTC', 'ATC', 'GCC', 'TTGG', 'pseknc.67', 'AAGA', 'TAAA', 'GT', 'ACGG', 'pseknc.59', 'GCGG', 'TGCG', 'GCTA', 'AGAA', 'ACAC', 'TAGT', 'TTA', 'GGCC', 'AAAC', 'pseknc.3', 'GTCC', 'GAAA', 'AGC', 'AAAG', 'TACG', 'GTT', 'TGAT', 'TGTA', 'AGTA', 'pseknc.72', 'CTAT', 'CT

# drop CDRSB_bl column

In [5]:
# raw_df = raw_df.drop(['CDRSB_bl'], axis=1)
# raw_df = raw_df.drop(['ADASQ4_bl'], axis=1)
# raw_df = raw_df.drop(['ADAS11_bl'], axis=1)
# raw_df = raw_df.drop(['ADAS13_bl'], axis=1)
# # features + target
# col_names = list(raw_df.columns)

# print(col_names), len(col_names)

# raw_df.head()

# shuffle the dataset

In [6]:
df = raw_df.sample(frac=1).reset_index(drop=True)
df.head()

AAA       GGG       AGT         C      CCCC        AC        TA  \
0  0.007912  0.024650  0.008217  0.357968  0.021613  0.054457  0.013082   
1  0.035489  0.010647  0.018927  0.173365  0.000394  0.047300  0.074892   
2  0.012017  0.030043  0.015451  0.245930  0.006014  0.048885  0.028302   
3  0.027073  0.023689  0.010152  0.266479  0.006772  0.051297  0.034386   
4  0.015716  0.025624  0.010933  0.279959  0.011278  0.045423  0.031079   

   pseknc.41       GGC       ACG  ...      ACCC      CTTT      GCCA      CCGA  \
0      0.017  0.027693  0.004260  ...  0.008828  0.002131  0.007915  0.002435   
1      0.005  0.009464  0.003549  ...  0.000789  0.010651  0.004339  0.001578   
2      0.019  0.030901  0.006009  ...  0.006014  0.006873  0.007732  0.003436   
3      0.015  0.024817  0.002820  ...  0.005643  0.008465  0.005643  0.001129   
4      0.013  0.021182  0.002733  ...  0.003759  0.005468  0.005810  0.001025   

        CTC       GTC      TGCC  pseknc.27      CACC  location  
0  0.028911  0.013086  0.008219      0.004  0.007915         1  
1  0.010647  0.005521  0.003945      0.003  0.000789         2  
2  0.016309  0.011159  0.001718      0.004  0.004296         0  
3  0.019741  0.011280  0.007901      0.003  0.006208         8  
4  0.027332  0.015374  0.006152      0.001  0.008202         1  

[5 rows x 151 columns]

In [7]:
# class frequency
from collections import Counter
print(sorted(Counter(df.location).items()))

[(0, 1504), (1, 1798), (2, 857), (3, 703), (4, 381), (5, 2752), (6, 180), (7, 156), (8, 1532)]


# oversample/undersample

In [8]:
# # oversample to balance the dataset
# ros = RandomUnderSampler(random_state=0)
# df_resampled, y_resampled = ros.fit_resample(df, df.location)

# # shufle the oversampled dataset
# df_resampled = df_resampled.sample(frac=1).reset_index(drop=True)
# df.head()

# # verify class frequency after oversampling
# from collections import Counter
# print(sorted(Counter(y_resampled).items()))


y_resampled = df.location
df_resampled = df

# manage column names

In [9]:
# list of the categorical and continuous columns
cat_names = []
cont_names = col_names.copy()

# remove the target columns' name    
cont_names.remove('location')

# remove the categorial columns' names
for cat_col in cat_names:
    cont_names.remove(cat_col)
cat_names, cont_names

([],
 ['AAA',
  'GGG',
  'AGT',
  'C',
  'CCCC',
  'AC',
  'TA',
  'pseknc.41',
  'GGC',
  'ACG',
  'pseknc.21',
  'GACG',
  'pseknc.11',
  'GG',
  'AAAA',
  'A',
  'AAT',
  'GCCC',
  'CCCA',
  'pseknc.0',
  'CAG',
  'pseknc.37',
  'CTA',
  'CC',
  'CGAT',
  'pseknc.65',
  'ACAG',
  'CCC',
  'ATT',
  'AA',
  'pseknc.15',
  'pseknc.42',
  'AT',
  'AAG',
  'AG',
  'pseknc.70',
  'CTCC',
  'AATT',
  'CGAA',
  'GAC',
  'G',
  'CGCG',
  'GGAG',
  'GA',
  'CCT',
  'AATG',
  'CGT',
  'pseknc.33',
  'AGTG',
  'GC',
  'GGCG',
  'CCGC',
  'GGT',
  'GAGC',
  'GCAG',
  'pseknc.18',
  'AATA',
  'GTCT',
  'ATTG',
  'pseknc.38',
  'GCT',
  'GGGG',
  'ACA',
  'pseknc.24',
  'CGTT',
  'ATTT',
  'TCG',
  'GAAG',
  'TGT',
  'CAC',
  'CGA',
  'CCTG',
  'ACGA',
  'GGGC',
  'TTC',
  'CT',
  'pseknc.68',
  'AACG',
  'TGC',
  'ATTA',
  'TTCG',
  'pseknc.73',
  'GAAC',
  'pseknc.9',
  'GTGA',
  'AGA',
  'GGTA',
  'GCA',
  'CAGA',
  'CCTC',
  'ATC',
  'GCC',
  'TTGG',
  'pseknc.67',
  'AAGA',
  'TAAA',
  'GT',


# convert categorical columns to one-hot encodings

In [10]:
df_ohe = df_resampled.copy()
for cat_col in cat_names:
    df_ohe = pd.get_dummies(df_ohe, columns=[cat_col])
df_ohe.head()

AAA       GGG       AGT         C      CCCC        AC        TA  \
0  0.007912  0.024650  0.008217  0.357968  0.021613  0.054457  0.013082   
1  0.035489  0.010647  0.018927  0.173365  0.000394  0.047300  0.074892   
2  0.012017  0.030043  0.015451  0.245930  0.006014  0.048885  0.028302   
3  0.027073  0.023689  0.010152  0.266479  0.006772  0.051297  0.034386   
4  0.015716  0.025624  0.010933  0.279959  0.011278  0.045423  0.031079   

   pseknc.41       GGC       ACG  ...      ACCC      CTTT      GCCA      CCGA  \
0      0.017  0.027693  0.004260  ...  0.008828  0.002131  0.007915  0.002435   
1      0.005  0.009464  0.003549  ...  0.000789  0.010651  0.004339  0.001578   
2      0.019  0.030901  0.006009  ...  0.006014  0.006873  0.007732  0.003436   
3      0.015  0.024817  0.002820  ...  0.005643  0.008465  0.005643  0.001129   
4      0.013  0.021182  0.002733  ...  0.003759  0.005468  0.005810  0.001025   

        CTC       GTC      TGCC  pseknc.27      CACC  location  
0  0.028911  0.013086  0.008219      0.004  0.007915         1  
1  0.010647  0.005521  0.003945      0.003  0.000789         2  
2  0.016309  0.011159  0.001718      0.004  0.004296         0  
3  0.019741  0.011280  0.007901      0.003  0.006208         8  
4  0.027332  0.015374  0.006152      0.001  0.008202         1  

[5 rows x 151 columns]

# normalize numerical features 

In [11]:
# normalize the continuous columns
for cont_name in cont_names:
    df_ohe[cont_name] = (df_ohe[cont_name] - min(df_ohe[cont_name]) )/(max(df_ohe[cont_name]) - min(df_ohe[cont_name]))

# verify that columns are normalized
print('max\tmin\tavg')
for cont_name in cont_names:
    print(f"{max(df_ohe[cont_name])}\t{min(df_ohe[cont_name])}\t{np.mean(df_ohe[cont_name])}")

max	min	avg
1.0	0.0	0.10823090721074899
1.0	0.0	0.22352060049934305
1.0	0.0	0.2560092839947923
1.0	0.0	0.4908204564115155
1.0	0.0	0.11761045440778853
1.0	0.0	0.29919424148465795
1.0	0.0	0.2512503632230287
1.0	0.0	0.20039446071557637
1.0	0.0	0.20573396024395518
1.0	0.0	0.14164303598676595
1.0	0.0	0.20170149226217263
1.0	0.0	0.08747861013261582
1.0	0.0	0.2772745230345449
1.0	0.0	0.2976881830136323
1.0	0.0	0.04772977285280691
1.0	0.0	0.4347216713603673
1.0	0.0	0.2725278583563302
1.0	0.0	0.18901117786447416
1.0	0.0	0.14855372027136185
1.0	0.0	0.09448754292184347
1.0	0.0	0.30340205941274606
1.0	0.0	0.24524463409581093
1.0	0.0	0.17140293714404942
1.0	0.0	0.36155115392170756
1.0	0.0	0.11606148348944291
1.0	0.0	0.3619969142868172
1.0	0.0	0.27511108147383173
1.0	0.0	0.21563867719060936
1.0	0.0	0.23229708391252232
1.0	0.0	0.23225080913035806
1.0	0.0	0.24722701003751146
1.0	0.0	0.21605340997756772
1.0	0.0	0.30567882182517786
1.0	0.0	0.2190255081925133
1.0	0.0	0.40151149076677617
1.0	0.0	0.2961507

In [12]:
# show the row count for the target labels (should be same now that the dataset is balanced)
[df_ohe.location.value_counts()[label] for label in range(9)]

[1504, 1798, 857, 703, 381, 2752, 180, 156, 1532]

In [13]:
# compute sample weights and class weights (unused)
sample_weights = compute_sample_weight(class_weight='balanced', y=df_ohe.location)
class_weights = compute_class_weight(class_weight='balanced', classes = np.unique(df_ohe.location), y=df_ohe.location)

class_weights_dict_label = dict(zip(np.unique(df_ohe.location),class_weights))
class_weights_dict_label_enc = dict(zip(range(3),class_weights))
class_weights

array([0.72864953, 0.60950439, 1.27875016, 1.55887466, 2.87634879,
       0.39821544, 6.0882716 , 7.02492877, 0.71533217])

In [14]:
# datasets for One-vs-All /pairwise classifications (used)

# df_ohe_CN_Demen = df_ohe.loc[df['DX'].isin(['CN','Dementia'])]
# # verify label-reassignment
# print(df_ohe_CN_Demen['DX'].unique(), df_ohe_MCI_Demen['DX'].unique(), df_ohe_CN_MCI['DX'].unique())

# # datasets for one vs all classifications
# df_ohe_CN_vs_all = df_ohe.copy()
# df_ohe_CN_vs_all = df_ohe_CN_vs_all.replace(['MCI','Dementia'],'Non')

# # verify label-reassignment
# print(df_ohe_CN_vs_all['DX'].unique(), df_ohe_MCI_vs_all['DX'].unique(), df_ohe_Demen_vs_all['DX'].unique())

# Metrics

In [15]:
# class for managing benchmark and performance metrics
class Metrics:
    
    def __init__(self, benchmark_acc = None, benchmark_f1 = None, benchmark_spec = None, benchmark_sen = None, benchmark_auc = None, benchmark_mcc = None):
        self.b_accuracy = benchmark_acc
        self.b_f1 = benchmark_f1
        self.b_sensitivity = benchmark_sen
        self.b_specificity = benchmark_spec
        self.b_auc_roc = benchmark_auc
        self.b_mcc = benchmark_mcc
        
    def reset_history(self):
        self.accuracies = []
        self.f1s = []
        self.recalls = []
        self.precisions = []
        self.sensitivity = []
        self.specificity = []
        self.auc_roc = []        
        self.tp_tn_fp_fn = []
        self.mcc = []

        
    metrics_names = ('acc','mcc',)# 'specificity', 'sensitivity', 'f1_score', 'auc_roc', 'recall', 
                     #'precision', 'mcc',  'tp','fp','tn','fn')
    @classmethod
    def compute_mcc(cls, y_true, y_pred):
        cm = confusion_matrix(y_true, y_pred)
        tn, fp, fn, tp = cm.ravel()
        mcc = (tp*tn - fp*fn) / (np.sqrt(  (tp+fp)*(tp+fn)*(tn+fp)*(tn+fn)  ) + 1e-8) 
        return mcc
    
    @classmethod
    def compute_sensitivity(cls, y_true, y_pred):
        cm = confusion_matrix(y_true, y_pred)
        tn, fp, fn, tp = cm.ravel()
        sensitivity = tp/(tp+fn)
        return sensitivity
    
    @classmethod
    def compute_specificity(cls, y_true, y_pred):
        cm = confusion_matrix(y_true, y_pred)
        tn, fp, fn, tp = cm.ravel()
        specificity = tn/(fp+tn)
        return specificity
    
    @classmethod
    def compute_accuracy(cls, y_true, y_pred):
        accuracy = (y_true==y_pred).sum()/len(y_true)
        return accuracy
    
    def compute_metrics(self, y_true, y_pred, label_list, epoch, do_print = False, store_vals = False):
        accuracy = (y_true==y_pred).sum()/len(y_true)
        mcc = matthews_corrcoef(y_true, y_pred)
        
        # for binary classification (unused)
        #cm = confusion_matrix(y_true, y_pred, labels = label_list)
        #tn, fp, fn, tp = cm.ravel()
        #specificity = tn/(fp+tn)
        #sensitivity = tp/(tp+fn)
        #f1 = f1_score(y_true, y_pred)
        #recall = recall_score(y_true, y_pred)
        #precision = precision_score(y_true, y_pred)
        #auc_roc = roc_auc_score( y_true, y_pred )  
        #mcc = (tp*tn - fp*fn) / (np.sqrt(  (tp+fp)*(tp+fn)*(tn+fp)*(tn+fn)  ) + 1e-8) 
        
        metrics_vals = ( accuracy, mcc, )# specificity, sensitivity, f1, auc_roc, recall, precision, mcc, tp, fp, tn, fn )

        results = ( metrics_names, metrics_vals )        
        
        return results
    
    

In [16]:
# metrics_names = Metrics.metrics_names
# clf_metrics = [ i+'_'+j for j in metrics_names for i in classifier_namelist+NN_labels ]
# clf_metrics

In [17]:
# np.array([[1,2],[3,4]]).transpose().reshape(1,-1)
# tmp, inner_CV_results_df

# Experiment params and configs

In [18]:
# parameters for NN training

NN_params = [ #{'solver': 'sgd', 'learning_rate': 'constant', 'momentum': 0, 'learning_rate_init': 0.2},
              #{'solver': 'sgd', 'learning_rate': 'constant', 'momentum': .9, 'nesterovs_momentum': False, 'learning_rate_init': 0.2},
              #{'solver': 'sgd', 'learning_rate': 'constant', 'momentum': .9, 'nesterovs_momentum': True, 'learning_rate_init': 0.2},
              #{'solver': 'sgd', 'learning_rate': 'invscaling', 'momentum': 0, 'learning_rate_init': 0.2},
              #{'solver': 'sgd', 'learning_rate': 'invscaling', 'momentum': .9, 'nesterovs_momentum': True, 'learning_rate_init': 0.2},
              #{'solver': 'sgd', 'learning_rate': 'invscaling', 'momentum': .9, 'nesterovs_momentum': False, 'learning_rate_init': 0.2},
              {'solver': 'adam', 'learning_rate_init': 0.01},
              #{'solver': 'adam', 'learning_rate': 'invscaling', 'learning_rate_init': 0.01}
            ]

# labels for NN classifier for different param configs
NN_labels = [ #"constant learning-rate", 
              #"constant with momentum",
              #"constant with Nesterov's momentum",
              #"inv-scaling learning-rate", 
              #"inv-scaling with momentum",
              #"inv-scaling with Nesterov's momentum", 
              #"NN_Adam",
             # "NN_Adam_inv_scaling_LR"
            ]

# maximum number of iterations for NN training
NN_max_iter = 100

# LightGBM training parameters
lgb_params  = {
          "objective" : "multiclass",
          "num_class" : 4,
          "num_leaves" : 60,
          "max_depth": -1,
          "learning_rate" : 0.01,
          "bagging_fraction" : 0.9,  # subsample
          "feature_fraction" : 0.9,  # colsample_bytree
          "bagging_freq" : 5,        # subsample_freq
          "bagging_seed" : 2018,
          "verbosity" : -1 }

# names of the classifiers (except for NN)
classifier_namelist = [ "RF"]# "RF"]#, "SVM", "LDA"]#, "XGBoost", "LightGBM"]

# setup the inner and outer k-fold 
num_split_outer = 5
num_split_inner = 5
skf_outer = StratifiedKFold(n_splits = num_split_outer)
skf_inner = StratifiedKFold(n_splits = num_split_inner)

# Format the data

In [19]:
# create the input matrix by dropping the target column from the dataset
X = (
        df_ohe
        .drop(['location'], axis=1)
        .astype(float)
    )

# the target column
y = df_ohe.location
label_list = sorted(y.unique()) 
label_list

[0, 1, 2, 3, 4, 5, 6, 7, 8]

# Label Encoding

In [20]:
# label encode the target variable
le = preprocessing.LabelEncoder()
le.fit(y)

y_label_enc = le.transform(y)
y_inv_str = le.inverse_transform(y_label_enc)

class_idx_to_name = dict(zip(range(len(le.classes_)),le.classes_))

# verify label encoding works as expected and show class label ordering for label encodings
np.unique(df_ohe.location), le.classes_, label_list, (y == y_inv_str).unique(), class_idx_to_name

(array([0, 1, 2, 3, 4, 5, 6, 7, 8]),
 array([0, 1, 2, 3, 4, 5, 6, 7, 8]),
 [0, 1, 2, 3, 4, 5, 6, 7, 8],
 array([ True]),
 {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8})

In [21]:
# Make sure the data doesn't have any NaNs (unused) 
#np.any(np.isnan(X)), np.all(np.isfinite(X)), type(df_ohe)
#df_ohe.to_csv("df_ohe.csv")


# SHAP Plots

In [22]:
# def save_shap_plots(model, X_test, class_idx_to_name):
    
#     if (isinstance(model, RandomForestClassifier)): 
        
#         classifier_name_abbr = 'rf'

#         start_time_shap = time.time()
#         explainer = shap.KernelExplainer(model.predict_proba, X_test)

#         with warnings.catch_warnings():
#             warnings.filterwarnings("ignore", module="shap")
#             shap_values = explainer.shap_values(X_test, nsamples = 40) #nsamples='auto')

#         end_time_shap = time.time()
#         print(f'Shap Elapsed Time: {end_time_shap - start_time_shap}')  
        
        
#     elif (isinstance(model, svm.SVC)):
        
#         classifier_name_abbr = 'svm'

#         start_time_shap = time.time()
#         explainer = shap.KernelExplainer(model.predict_proba, X_test, link="logit")
    
#         with warnings.catch_warnings():
#             warnings.filterwarnings("ignore", module="shap")
#             shap_values = explainer.shap_values(X_test, nsamples = 30)

#         end_time_shap = time.time()
#         print(f'Shap Elapsed Time: {end_time_shap - start_time_shap}')        
        
#     elif (isinstance(model, LinearDiscriminantAnalysis)):
    
#         classifier_name_abbr = 'lda'

#         start_time_shap = time.time()
#         explainer = shap.KernelExplainer(model.predict_proba, X_test, link="logit")
        
#         with warnings.catch_warnings():
#             warnings.filterwarnings("ignore", module="shap")
#             shap_values = explainer.shap_values(X_test, nsamples=100)

#         end_time_shap = time.time()
#         print(f'Shap Elapsed Time: {end_time_shap - start_time_shap}')        
        
        
#     elif (isinstance(model, xgboost.XGBClassifier)):
    
#         classifier_name_abbr = 'xgboost'

#         start_time_shap = time.time()
#         explainer = shap.TreeExplainer(model)

#         with warnings.catch_warnings():
#             warnings.filterwarnings("ignore", module="shap")
#             shap_values = explainer.shap_values(X_test)

#         end_time_shap = time.time()
#         print(f'Shap Elapsed Time: {end_time_shap - start_time_shap}')   
        
        
        
#     elif (isinstance(model, lgb.LGBMClassifier)):
    
#         classifier_name_abbr = 'lightGBM'

#         start_time_shap = time.time()
#         explainer = shap.TreeExplainer(model)

#         with warnings.catch_warnings():
#             warnings.filterwarnings("ignore", module="shap")
#             shap_values = explainer.shap_values(X_test)

#         end_time_shap = time.time()
#         print(f'Shap Elapsed Time: {end_time_shap - start_time_shap}')
 
    
#     elif (isinstance(model, MLPClassifier)):
        
#         classifier_name_abbr = 'NN'

#         start_time_shap = time.time()
#         explainer = shap.KernelExplainer(model.predict_proba, X_test)

#         with warnings.catch_warnings():
#             warnings.filterwarnings("ignore", module="shap")
#             shap_values = explainer.shap_values(X_test, nsamples=80)

#         end_time_shap = time.time()
#         print(f'Shap Elapsed Time: {end_time_shap - start_time_shap}')

    
#     # load JS visualization code to notebook
#     shap.initjs()
        
#     # Generate the summary plot, the force plots and the dependence plots
    
# #     f = plt.figure()
# #     shap.summary_plot(shap_values, X_test)
# #     f.savefig(f"../figs/{classifier_name_abbr}/summary_plot_{classifier_name_abbr}.png", bbox_inches='tight', dpi=600) 

# #     for class_idx in class_idx_to_name.keys():
# #         print(class_idx_to_name[class_idx])
# #         #f = plt.figure()
# #         shap.force_plot(explainer.expected_value[class_idx], shap_values[class_idx], X_test)
# #         #f.savefig(f"figs/{classifier_name_abbr}/force_plot_{classifier_name_abbr}_{class_idx}_{class_idx_to_name[class_idx]}.png", bbox_inches='tight', dpi=600)

# #     for class_idx in class_idx_to_name.keys():
# #         print(class_idx_to_name[class_idx])
# #         for colname in X_test.columns:
# #             #f = plt.figure()

In [23]:
# #             shap.dependence_plot(colname, shap_values[class_idx], X_test)   
# #             #f.savefig(f"figs/{classifier_name_abbr}/dependence_plot_{classifier_name_abbr}_{class_idx}_{class_idx_to_name[class_idx]}_col-{colname}.png", bbox_inches='tight', dpi=600)             

#     # hack for displaying force_plots by hand
#     return (explainer, shap_values, X_test)


# Training

In [33]:
pd.options.display.float_format = '{:,.6f}'.format

# for holding the results of validations in the inner folds
all_CV_results = []
all_CV_confusion_mats = []

# get the names of all used metrics 
metrics = Metrics()
metrics_names = Metrics.metrics_names

# result table header formation
clf_metrics_labels = [ i+'_'+j for j in metrics_names for i in classifier_namelist+NN_labels ]

# for holding the results of testing on the outer folds
outer_CV_results_df = pd.DataFrame({**dict.fromkeys(clf_metrics_labels, [])})
outer_CV_confusion_mats = []

show_header = True

outer_fold_idx = 0

# outer k-fold (for generalization performance estimation)
for train_idx_outer, test_idx_outer in skf_outer.split(X, y_label_enc):

    outer_fold_idx += 1
    
    X_train_outer = X.iloc[train_idx_outer]
    y_train_outer = y_label_enc[train_idx_outer]
    sample_weights_train_outer = sample_weights[train_idx_outer]
    
    X_test_outer = X.iloc[test_idx_outer]
    y_test_outer = y_label_enc[test_idx_outer]  
    #class_weights_test_outer = sample_weights[test_idx_outer]
    
    inner_split_idx = 0
    
    # dataframe for holding the results from all the inner folds for all the candidate classifiers 
    inner_CV_results_df = pd.DataFrame({**dict.fromkeys(clf_metrics_labels, []) })
    inner_CV_fold_conf_mats = []
    
    start_time = time.time()
    
    # inner k-fold (for comparing model performance/hyper parameter tuning)
    for train_idx_inner, test_idx_inner in skf_inner.split(X_train_outer, y_train_outer):
        
        inner_split_idx +=1
        
        # inner fold's trainset
        X_train_inner = X_train_outer.iloc[train_idx_inner]
        y_train_inner = y_train_outer[train_idx_inner]
        sample_weights_train_inner = sample_weights_train_outer[train_idx_inner]

        #print(len(y_train_inner))
        # oversample to balance the dataset
        ros = RandomOverSampler(random_state=0)
        #X_train_inner, y_train_inner = ros.fit_resample(X_train_inner, y_train_inner)
        
        
        
        #print(len(y_train_inner))
        
        # shufle the oversampled dataset
        #X_train_inner = X_train_inner.sample(frac=1).reset_index(drop=True)
        #df.head()
        
        # verify class frequency after oversampling
        from collections import Counter
        #print(sorted(Counter(y_train_inner).items()))
        

        # inner fold's testset
        X_test_inner = X_train_outer.iloc[test_idx_inner]
        y_test_inner = y_train_outer[test_idx_inner]
        #sample_weights_test_inner = sample_weights_train_outer[test_idx_inner]
        
        
        # initialize the classifiers
        #rf_clf = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 42)#, class_weight = class_weights_dict_label_enc)
        #svm_clf = svm.SVC(kernel='rbf', probability=True)#, class_weight = class_weights_dict_label_enc)
        #lda_clf = LinearDiscriminantAnalysis()
        xgb = xgboost.XGBClassifier()
        #lgb_clf = lgb.LGBMClassifier(**lgb_params)
        
        
        # list of classifiers to iterate over (except for NN)
        classifiers = [xgb]#rf_clf]#, svm_clf, lda_clf]#, xgb, lgb_clf ]

        ova_classifiers = [OneVsOneClassifier(clf) for clf in classifiers]    
    
        
        # add the NNs to the list    
        #for NN_config in NN_params:
        #    classifiers.append(MLPClassifier(alpha=1e-5, hidden_layer_sizes=(100, 75, 25, 16), random_state=1, **NN_config))
        
        # to hold the results from the classifiers in each inner fold        
        clf_results_inner_fold = []
        clf_conf_mats_inner_fold = []
        
        # iterate over the classifiers to compute their performance on a specific inner fold
        for clf in ova_classifiers:
            
            clf_start_time = time.time()
            
            #print(type(clf))
            
            #mean_acc = 0.0
            
            # some parameter combinations will not converge as can be seen on the
            # plots so they are ignored here
            with warnings.catch_warnings():
                warnings.filterwarnings("ignore", category=ConvergenceWarning,
                                        module="sklearn")
            
                # train the candidate classifier
                clf.fit(X_train_inner, y_train_inner)

            #y_pred_probs = clf.predict_proba(X_test_inner)
            #y_pred_probs = clf.predict_proba(X_test.iloc[0:1])
            y_pred_inner = clf.predict(X_test_inner)

            # compute the mean accuracy
            # mean_acc = clf.score(X_test_inner, y_test_inner)

            _, results_metrics = metrics.compute_metrics(y_test_inner, y_pred_inner, label_list, -1)
            confusion_mat = confusion_matrix(y_test_inner, y_pred_inner, normalize = 'true')            
            
            #print(confusion_mat)

            # store the result from the classifier for this inner fold
            clf_results_inner_fold.append(results_metrics)
            clf_conf_mats_inner_fold.append(confusion_mat)
            print('inner fold result:')
            print(confusion_mat)
            print('results:',results_metrics)
            print()

            clf_end_time = time.time()
            print(f'Elapsed time: {clf_end_time-clf_start_time}')
        
        print('done training one inner fold clf')
        # create a tmp dataframe containing the results from different classifiers on this inner fold
        tmp = pd.DataFrame(np.array(clf_results_inner_fold).transpose().reshape(1,-1), columns=inner_CV_results_df.columns )        
        
        # update the results
        inner_CV_results_df = inner_CV_results_df.append(tmp)
        inner_CV_fold_conf_mats.append(clf_conf_mats_inner_fold)
        
    # store the result for the outer fold
    all_CV_results.append(inner_CV_results_df)       
    all_CV_confusion_mats.append(inner_CV_fold_conf_mats)
    print('after inner fold')
    
    # for re-training the classifiers on the outer fold's entire trainset
    # initialize the classifiers
    #rf_clf = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 42)#, class_weight = class_weights_dict_label_enc)
    #svm_clf = svm.SVC(kernel='rbf', probability=True)#, class_weight = class_weights_dict_label_enc)
    #lda_clf = LinearDiscriminantAnalysis()
    xgb = xgboost.XGBClassifier()
    #lgb_clf = lgb.LGBMClassifier(**lgb_params)

    # list of classifiers to iterate over (except for NN)
    classifiers = [xgb]#rf_clf]#, svm_clf, lda_clf]#, xgb, lgb_clf ]

    ova_classifiers = [OneVsOneClassifier(clf) for clf in classifiers]    

    # add the NNs to the list    
    #for NN_config in NN_params:
    #    classifiers.append(MLPClassifier(alpha=1e-5, hidden_layer_sizes=(100, 75, 25, 16), random_state=1, **NN_config))
    
    # to hold the results from the classifiers in each outer fold        
    clf_results_outer_fold = []
    confusion_mats_outer_fold = []    
        
    for clf in ova_classifiers:

        clf_start_time = time.time()
        
        mean_acc = 0.0
            
        # some parameter combinations will not converge as can be seen on the
        # plots so they are ignored here
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore", category=ConvergenceWarning,
                                    module="sklearn")
            # train the candidate classifier
            #if (isinstance(clf, xgboost.XGBClassifier)):
            #    clf.fit(X_train_inner, y_train_inner, sample_weight = sample_weights_train_inner)
            #else:
            
            ros = RandomOverSampler(random_state=0)
            #X_train_outer, y_train_outer = ros.fit_resample(X_train_outer, y_train_outer)
            
            #print(len(y_train_inner))

            # shufle the oversampled dataset
            #X_train_inner = X_train_inner.sample(frac=1).reset_index(drop=True)
            #df.head()

            # verify class frequency after oversampling
            from collections import Counter
            #print(sorted(Counter(y_train_outer).items()))

            
            
            clf.fit(X_train_outer, y_train_outer)
            
            #y_pred_probs = clf.predict_proba(X_test_outer)
            #y_pred_probs = clf.predict_proba(X_test.iloc[0:1])
            y_pred_outer = clf.predict(X_test_outer)

            # compute the mean accuracy
            # mean_acc = clf.score(X_test_inner, y_test_inner)

            #print()
            _, results_metrics = metrics.compute_metrics(y_test_outer, y_pred_outer, label_list, -1)
            outer_confusion_mat = confusion_matrix(y_test_outer, y_pred_outer)
            
            #if (outer_fold_idx == 1) and isinstance(clf, xgboost.XGBClassifier):
            #    print(type(clf))
            #    (explainer, shap_values, X_test) = save_shap_plots(clf, X_test_outer, class_idx_to_name)
                 
                    
            #print()

        # store the result from the classifier for this inner fold
        #print(clf_results_outer_fold)
        #print()
        clf_results_outer_fold.append(results_metrics)
        #print(clf_results_outer_fold)
        #print()
        #print()
        confusion_mats_outer_fold.append(outer_confusion_mat)
        print('Outer fold result:')
        print(outer_confusion_mat)
        print('results:',results_metrics)
            
        print()
        clf_end_time = time.time()
        print(f'Elapsed time: {clf_end_time-clf_start_time}')

    print(clf_results_outer_fold)
    print()
    # store the results from the outer fold
    tmp = pd.DataFrame(np.array(clf_results_outer_fold).transpose().reshape(1,-1), columns=outer_CV_results_df.columns )
    
    # append the results from this outer fold to the collection
    outer_CV_results_df = outer_CV_results_df.append(tmp)
    outer_CV_confusion_mats.append(confusion_mats_outer_fold)
    
    # display the results from this outer fold
    print(outer_CV_results_df.tail(1).to_string(index=False, header= show_header), end="\t")
    show_header = False

    end_time = time.time()
    print(f'Elapsed time: {end_time-start_time}')
    
    

/home/tarik/anaconda3/envs/testbed/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[12:50:52] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/home/tarik/anaconda3/envs/testbed/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[12:50:53] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/home/tarik/anaconda3/envs/testbed/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


...

/home/tarik/anaconda3/envs/testbed/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[12:51:44] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/home/tarik/anaconda3/envs/testbed/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/home/tarik/anaconda3/envs/testbed/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[12:51:45] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:51:45] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


KeyboardInterrupt: 

# Outer folds' results

In [31]:
outer_CV_results_df.head(10)

RF_acc   RF_mcc
0 0.525089 0.417055
0 0.516979 0.406478
0 0.527116 0.419002
0 0.517748 0.407683
0 0.525862 0.418456

In [32]:
clf_results_outer_fold

[(0.5258620689655172, 0.41845580963473666)]

# Mean result across outer folds

In [27]:
# clf_results_inner_fold, ['Random Forest','SVM', 'LDA', *NN_labels], len(NN_params), len(classifiers)
outer_CV_results_df.mean(axis = 0)

RF_acc   0.476733
RF_mcc   0.355490
dtype: float64

# Find which classifier performed best in each outer fold

In [28]:
for fold_i, all_CV_result in enumerate(all_CV_results):
    print(f"Fold: {fold_i+1:02d} Best classifier name: {type(classifiers[np.argmax(all_CV_result.mean(axis = 0))]).__name__}, index:{np.argmax(all_CV_result.mean(axis = 0))}" )

Fold: 01 Best classifier name: RandomForestClassifier, index:0
Fold: 02 Best classifier name: RandomForestClassifier, index:0
Fold: 03 Best classifier name: RandomForestClassifier, index:0
Fold: 04 Best classifier name: RandomForestClassifier, index:0
Fold: 05 Best classifier name: RandomForestClassifier, index:0


# confusion matrix of selected classifier averaged across outer folds 

In [29]:
selected_classifier_idx = 3

np.set_printoptions(precision=4, suppress=True)
outer_CV_confusion_mats_np = np.array(outer_CV_confusion_mats)
#print('outer_CV_confusion_mats_np.shape:',outer_CV_confusion_mats_np.shape)
conf_mat_freq_meaned = outer_CV_confusion_mats_np[:,selected_classifier_idx,:,:].mean(axis = 0)
#print(conf_mat_freq_meaned)
conf_mat_pct_meaned = conf_mat_freq_meaned/conf_mat_freq_meaned.sum(axis =1)
print()
print(f"conf_mat_pct_meaned\n{conf_mat_pct_meaned}")

IndexError: index 3 is out of bounds for axis 1 with size 1

# inner folds' (mean) results

In [ ]:
[print(inner_CV_result.mean(axis = 0)) for inner_CV_result in all_CV_results]

# Generate the summary plot

In [ ]:
# load JS visualization code to notebook
shap.initjs()

classifier_name_abbr = 'xgboost'

f = plt.figure()
shap.summary_plot(shap_values, X_test)
f.savefig(f"../figs/{classifier_name_abbr}/summary_plot_{classifier_name_abbr}.png", bbox_inches='tight', dpi=600) 

# Generate the force plots

In [ ]:
for class_idx in class_idx_to_name.keys():
    print(class_idx_to_name[class_idx])
    #f = plt.figure()
    shap.force_plot(explainer.expected_value[class_idx], shap_values[class_idx], X_test)
    #f.savefig(f"figs/{classifier_name_abbr}/force_plot_{classifier_name_abbr}_{class_idx}_{class_idx_to_name[class_idx]}.png", bbox_inches='tight', dpi=600)

# Generate the foce plots (model output value)

In [ ]:
print(class_idx_to_name[0])
shap.force_plot(explainer.expected_value[0], shap_values[0], X_test)

In [ ]:
print(class_idx_to_name[1])
shap.force_plot(explainer.expected_value[1], shap_values[1], X_test)

In [ ]:
print(class_idx_to_name[2])
shap.force_plot(explainer.expected_value[2], shap_values[2], X_test)

# Generate the foce plots (variable's effects)

In [ ]:
print(class_idx_to_name[0])
shap.force_plot(explainer.expected_value[0], shap_values[0], X_test)

In [ ]:
print(class_idx_to_name[1])
shap.force_plot(explainer.expected_value[1], shap_values[1], X_test)

In [ ]:
print(class_idx_to_name[2])
shap.force_plot(explainer.expected_value[2], shap_values[2], X_test)

# Generate the dependence plots

In [ ]:
for class_idx in class_idx_to_name.keys():
    print('-------------------------------------------------------------------')    
    print(class_idx_to_name[class_idx])
    print('-------------------------------------------------------------------')
    for colname in X_test.columns:
        if colname == 'CDRSB_bl':
            print('****************')
            print(f"{colname} ({class_idx_to_name[class_idx]})")
            print('****************')
            #f = plt.figure()
            shap.dependence_plot(colname, shap_values[class_idx], X_test)   
            #f.savefig(f"figs/{classifier_name_abbr}/dependence_plot_{classifier_name_abbr}_{class_idx}_{class_idx_to_name[class_idx]}_col-{colname}.png", bbox_inches='tight', dpi=600)     

In [ ]:
colname = 'MMSE_bl'
class_idx = 0
print('****************')
print(f"{colname} ({class_idx_to_name[class_idx]})")
print('****************')
#f = plt.figure()
shap.dependence_plot(colname, shap_values[class_idx], X_test) 

In [ ]:
colname = 'MMSE_bl'
class_idx = 1
print('****************')
print(f"{colname} ({class_idx_to_name[class_idx]})")
print('****************')
#f = plt.figure()
shap.dependence_plot(colname, shap_values[class_idx], X_test) 

In [ ]:
colname = 'MMSE_bl'
class_idx = 2
print('****************')
print(f"{colname} ({class_idx_to_name[class_idx]})")
print('****************')
#f = plt.figure()
shap.dependence_plot(colname, shap_values[class_idx], X_test) 